In [1]:
import numpy as np
import datajoint as dj
import time
import os
import datetime
from tqdm import tqdm

#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

schema = dj.schema('microns_pinky')
pinky = dj.create_virtual_module('pinky', 'microns_pinky')
pinky_nda = dj.create_virtual_module('pinky_nda','microns_pinky_nda')
ta3p100 = dj.create_virtual_module("ta3p100","microns_ta3p100")

Connecting celiib@10.28.0.34:3306


In [9]:
@schema
class NeuriteTargetsRevised(dj.Computed):
    definition = """
    -> pinky.Neurite
    ---
    apical_num           : float                        # total number of apicals synapsis for neurite
    apical_prop          : float                        # perc of total number of apicals synapsis for neurite
    apical_spine_prop    : float                        # perc of apical synapses that are apical spine synapses
    apical_spine_head_prop : float                        # perc of apical synapses that are apical spine head synapses
    apical_spine_neck_prop : float                        # perc of apical synapses that are apical spine neck synapses
    apical_shaft_prop    : float                        # perc of apical synapses that are apical shaft synapses
    basal_num            : float                        # total number of basal synapsis for neurite
    basal_prop           : float                        # perc of total number of basal synapsis for neurite
    basal_spine_prop     : float                        # perc of basal synapses that are basal spine synapses
    basal_spine_head_prop : float                        # perc of basal synapses that are basal spine head synapses
    basal_spine_neck_prop : float                        # perc of basal synapses that are basal spine neck synapses
    basal_shaft_prop     : float                        # perc of basal synapses that are basal shaft synapses
    oblique_num          : float                        # total number of oblique synapsis for neurite
    oblique_prop         : float                        # perc of total number of oblique synapsis for neurite
    oblique_spine_prop   : float                        # perc of oblique synapses that are oblique spine synapses
    oblique_spine_head_prop : float                        # perc of oblique synapses that are oblique spine head synapses
    oblique_spine_neck_prop : float                        # perc of oblique synapses that are oblique spine neck synapses
    oblique_shaft_prop   : float                        # perc of oblique synapses that are oblique shaft synapses
    dendrite_num         : float                        # total number of dendrite synapsis for neurite
    dendrite_prop        : float                        # perc of total number of dendrite synapsis for neurite
    dendrite_spine_prop  : float                        # perc of dendrite synapses that are dendrite spine synapses
    dendrite_spine_head_prop : float                        # perc of dendrite synapses that are dendrite spine head synapses
    dendrite_spine_neck_prop : float                        # perc of dendrite synapses that are dendrite spine neck synapses
    dendrite_shaft_prop  : float                        # perc of dendrite synapses that are dendrite shaft synapse
    soma_num             : float                        # total number of soma synapsis for neurite
    soma_prop            : float                        # perc of total number of soma synapsis for neurite
    soma_non_spine_prop  : float                        # perc of total number of soma non_spine synapse
    axon_num             : float                        # total number of axon synapsis for neurite
    axon_prop            : float                        # perc of total number of axon synapsis for neurite
    axon_non_spine_prop  : float                        # perc of total number of axon non_spine synapse
    distal_num           : float                        # total number of distal synapsis for neurite
    distal_prop          : float                        # perc of total number of distal synapsis for neurite
    distal_non_spine_prop : float                        # perc of total number of distal non_spine synapse
    cilia_num            : float                        # total number of cilia synapsis for neurite
    cilia_prop           : float                        # perc of total number of cilia synapsis for neurite
    cilia_non_spine_prop : float                        # perc of total number of cilia non_spine synapse
    spine_num            : float                        # total number of spine synapsis for neurite
    spine_prop           : float                        # perc of total number of spine synapsis for neurite
    spine_head_num       : float                        # total number of spine head synapsis for neurite
    spine_head_prop      : float                        # perc of total number of spine head synapsis for neurite
    spine_neck_num       : float                        # total number of spine neck synapsis for neurite
    spine_neck_prop      : float                        # perc of total number of spine neck synapsis for neurite
    shaft_num            : float                        # total number of shaft synapsis for neurite
    shaft_prop           : float                        # perc of total number of shaft synapsis for neurite
    non_spine_num        : float                        # total number of non-spinable synapsis (aka not on apic/dendr/basal/oblique)
    non_spine_prop       : float                        # perc of total number of non-spinable synapsis (aka not on apic/dendr/basal/oblique)
    total                : float                        # total number of synapsis for neurite
    """

    key_source = pinky.CurrentSegmentation()
    
    def make(self, key):

        #The way did NeuriteTargets does it
        fingerprints = dj.U('segmentation','segment_id').aggr((pinky.SynapseCompartmentSpineLabelRevised & pinky.CurrentSegmentation()), 
                            apical_num='sum(compartment_label = 2)',
                            apical_prop='sum(compartment_label = 2)/count(*)',
                            apical_spine_prop='sum(spine_label = 14 AND compartment_label = 2)/count(compartment_label = 2)',
                            apical_spine_head_prop='sum(spine_label = 13 AND compartment_label = 2)/count(compartment_label = 2)',
                            apical_spine_neck_prop='sum(spine_label = 15 AND compartment_label = 2)/count(compartment_label = 2)',
                            apical_shaft_prop='sum(spine_label = 0 AND compartment_label = 2)/count(compartment_label = 2)',  

                            basal_num='sum(compartment_label = 3)',
                            basal_prop='sum(compartment_label = 3)/count(*)',
                            basal_spine_prop='sum(spine_label = 14 AND compartment_label = 3)/count(compartment_label = 3)',
                            basal_spine_head_prop='sum(spine_label = 13 AND compartment_label = 3)/count(compartment_label = 3)',
                            basal_spine_neck_prop='sum(spine_label = 15 AND compartment_label = 3)/count(compartment_label = 3)',
                            basal_shaft_prop='sum(spine_label = 0 AND compartment_label = 3)/count(compartment_label = 3)', 

                            oblique_num='sum(compartment_label = 4)',
                            oblique_prop='sum(compartment_label = 4)/count(*)',
                            oblique_spine_prop='sum(spine_label = 14 AND compartment_label = 4)/count(compartment_label = 4)',
                            oblique_spine_head_prop='sum(spine_label = 13 AND compartment_label = 4)/count(compartment_label = 4)',
                            oblique_spine_neck_prop='sum(spine_label = 15 AND compartment_label = 4)/count(compartment_label = 4)',
                            oblique_shaft_prop='sum(spine_label = 0 AND compartment_label = 4)/count(compartment_label = 4)', 

                            dendrite_num='sum(compartment_label = 8)',
                            dendrite_prop='sum(compartment_label = 8)/count(*)',
                            dendrite_spine_prop='sum(spine_label = 14 AND compartment_label = 8)/count(compartment_label = 8)',
                            dendrite_spine_head_prop='sum(spine_label = 13 AND compartment_label = 8)/count(compartment_label = 8)',
                            dendrite_spine_neck_prop='sum(spine_label = 15 AND compartment_label = 8)/count(compartment_label = 8)',
                            dendrite_shaft_prop='sum(spine_label = 0 AND compartment_label = 8)/count(compartment_label = 8)', 

                            soma_num='sum(compartment_label = 5)',
                            soma_prop='sum(compartment_label = 5)/count(*)',
                            soma_non_spine_prop='sum(spine_label = 18 AND compartment_label = 5)/count(compartment_label = 5)',
                            #soma_non_spine_prop='sum(spine_label = 18 AND compartment_label = 5)/count(compartment_label = 5)',                                       
                            #soma_spine_prop='sum(spine_label = 14 AND compartment_label = 5)/count(compartment_label = 5)',
                            #soma_spine_head_prop='sum(spine_label = 13 AND compartment_label = 5)/count(compartment_label = 5)',
                            #soma_spine_neck_prop='sum(spine_label = 15 AND compartment_label = 5)/count(compartment_label = 5)',
                            #soma_shaft_prop='sum(spine_label = 0 AND compartment_label = 5)/count(compartment_label = 5)', 

                            axon_num='sum(compartment_label = 6 OR compartment_label = 7)',
                            axon_prop='sum(compartment_label = 6 OR compartment_label = 7)/count(*)',
                            axon_non_spine_prop='sum(spine_label = 18 AND compartment_label = 6 OR spine_label = 18 AND compartment_label = 7)/count(compartment_label = 6 OR compartment_label = 7)',

                            #axon_non_spine_prop='sum(spine_label = 18 AND compartment_label = 6)/count(compartment_label = 5)',


                            distal_num='sum(compartment_label = 9)',
                            distal_prop='sum(compartment_label = 9)/count(*)',
                            distal_non_spine_prop='sum(spine_label = 18 AND compartment_label = 9)/count(compartment_label = 9)',

                            cilia_num='sum(compartment_label = 12)',
                            cilia_prop='sum(compartment_label = 12)/count(*)',
                            cilia_non_spine_prop='sum(spine_label = 18 AND compartment_label = 12)/count(compartment_label = 12)',



                            spine_num='sum(spine_label = 14)',
                            spine_prop='sum(spine_label = 14)/count(*)',
                            spine_head_num='sum(spine_label = 13)',
                            spine_head_prop='sum(spine_label = 13)/count(*)',
                            spine_neck_num='sum(spine_label = 15)',
                            spine_neck_prop='sum(spine_label = 15)/count(*)',
                            shaft_num='sum(spine_label = 0)',
                            shaft_prop='sum(spine_label = 0)/count(*)',
                            non_spine_num='sum(spine_label = 18)',
                            non_spine_prop='sum(spine_label = 18)/count(*)',
                            total='count(*)')
        self.insert(fingerprints)

In [10]:
import time
start_time = time.time()
NeuriteTargetsRevised.populate()
print(f"Total time = " + str(time.time() - start_time))

Total time = 5.6618170738220215


In [5]:
NeuriteTargetRevised() & "dendrite_spine_neck_prop>0"

segmentation segmentation id,segment_id segment id unique within each Segmentation,apical_num total number of apicals synapsis for neurite,apical_prop perc of total number of apicals synapsis for neurite,apical_spine_prop perc of apical synapses that are apical spine synapses,apical_spine_head_prop perc of apical synapses that are apical spine head synapses,apical_spine_neck_prop perc of apical synapses that are apical spine neck synapses,apical_shaft_prop perc of apical synapses that are apical shaft synapses,basal_num total number of basal synapsis for neurite,basal_prop perc of total number of basal synapsis for neurite,basal_spine_prop perc of basal synapses that are basal spine synapses,basal_spine_head_prop perc of basal synapses that are basal spine head synapses,basal_spine_neck_prop perc of basal synapses that are basal spine neck synapses,basal_shaft_prop perc of basal synapses that are basal shaft synapses,oblique_num total number of oblique synapsis for neurite,oblique_prop perc of total number of oblique synapsis for neurite,oblique_spine_prop perc of oblique synapses that are oblique spine synapses,oblique_spine_head_prop perc of oblique synapses that are oblique spine head synapses,oblique_spine_neck_prop perc of oblique synapses that are oblique spine neck synapses,oblique_shaft_prop perc of oblique synapses that are oblique shaft synapses,dendrite_num total number of dendrite synapsis for neurite,dendrite_prop perc of total number of dendrite synapsis for neurite,dendrite_spine_prop perc of dendrite synapses that are dendrite spine synapses,dendrite_spine_head_prop perc of dendrite synapses that are dendrite spine head synapses,dendrite_spine_neck_prop perc of dendrite synapses that are dendrite spine neck synapses,dendrite_shaft_prop perc of dendrite synapses that are dendrite shaft synapse,soma_num total number of soma synapsis for neurite,soma_prop perc of total number of soma synapsis for neurite,soma_non_spine_prop perc of total number of soma non_spine synapse,axon_num total number of axon synapsis for neurite,axon_prop perc of total number of axon synapsis for neurite,axon_non_spine_prop perc of total number of axon non_spine synapse,distal_num total number of distal synapsis for neurite,distal_prop perc of total number of distal synapsis for neurite,distal_non_spine_prop perc of total number of distal non_spine synapse,cilia_num total number of cilia synapsis for neurite,cilia_prop perc of total number of cilia synapsis for neurite,cilia_non_spine_prop perc of total number of cilia non_spine synapse,spine_num total number of spine synapsis for neurite,spine_prop perc of total number of spine synapsis for neurite,spine_head_num total number of spine head synapsis for neurite,spine_head_prop perc of total number of spine head synapsis for neurite,spine_neck_num total number of spine neck synapsis for neurite,spine_neck_prop perc of total number of spine neck synapsis for neurite,shaft_num total number of shaft synapsis for neurite,shaft_prop perc of total number of shaft synapsis for neurite,non_spine_num total number of non-spinable synapsis (aka not on apic/dendr/basal/oblique),non_spine_prop perc of total number of non-spinable synapsis (aka not on apic/dendr/basal/oblique),total total number of synapsis for neurite
3,648518346341351641,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,1.0,0.0,0.8,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.8,1.0,0.2,0.0,0.0,0.0,0.0,5.0
3,648518346341351695,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.333333,0.0,0.222222,0.111111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.666667,0.0,0.444444,0.111111,0.111111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.666667,2.0,0.222222,1.0,0.111111,0.0,0.0,9.0
3,648518346341352081,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.454545,0.0909091,0.0909091,0.0,0.272727,1.0,0.0909091,0.0,0.0909091,0.0,0.0,5.0,0.454545,0.0,0.181818,0.0909091,0.181818,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0909

In [ ]:
pinky.SynapseCompartmentSpineLabelRevised() & "spine_label=0"